In [1]:
#!pip install glfw
#!pip install pyopengl
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import random

In [2]:


def f(altura, raio, qtd, rotacao_inicial=0):
    angulos = np.linspace(0+rotacao_inicial, 2 * np.pi + rotacao_inicial, qtd, endpoint=False)
    pontos = [(raio * np.cos(ang), altura,raio * np.sin(ang)) for ang in angulos]
    return pontos

quad_inf = f(0,0.5,4, np.pi/4)
quad_sup = f(1,0.5,4, np.pi/4)

#cubo
objeto_1 = [
    # Face inferior (base do cubo)
    quad_inf[0], quad_inf[3], quad_inf[2],  # invertido
    quad_inf[0], quad_inf[2], quad_inf[1],

    # Face superior (topo do cubo)
    quad_sup[0], quad_sup[2], quad_sup[3],
    quad_sup[0], quad_sup[1], quad_sup[2],  # invertido

    # Face lateral 1 (entre quad_inf[0] e quad_sup[0])
    quad_inf[0], quad_inf[1], quad_sup[0],  # invertido
    quad_inf[1], quad_sup[1], quad_sup[0],

    # Face lateral 2 (entre quad_inf[1] e quad_sup[1])
    quad_inf[1], quad_inf[2], quad_sup[1],  # invertido
    quad_inf[2], quad_sup[2], quad_sup[1],

    # Face lateral 3 (entre quad_inf[2] e quad_sup[2])
    quad_inf[2], quad_inf[3], quad_sup[2],  # invertido
    quad_inf[3], quad_sup[3], quad_sup[2],

    # Face lateral 4 (entre quad_inf[3] e quad_sup[3])
    quad_inf[3], quad_inf[0], quad_sup[3],  # invertido
    quad_inf[0], quad_sup[0], quad_sup[3]
]



#diamanate
quad= f(1,1/5, 4, np.pi/4)
octa= f(2/3,2/5, 8,np.pi/4)
zero = (0,0,0)

objeto_2 = [ 
    #abc
    quad[0],quad[2],quad[1],
    quad[2],quad[0],quad[3],
    
    quad[0], octa[1], octa[0],
    quad[0], quad[1], octa[1],
    quad[1],octa[2],octa[1],
    quad[1], octa[3], octa[2],
    quad[1], quad[2], octa[3],
    quad[2], octa[4], octa[3],
    quad[2],octa[5],octa[4],
    quad[2],quad[3],octa[5],
    quad[3],octa[6],octa[5],
    quad[3], octa[7], octa[6],
    quad[3],quad[0],octa[7],
    quad[0],octa[0],octa[7],

    octa[0],octa[1],zero,
    octa[1],octa[2],zero,
    octa[2],octa[3],zero,
    octa[3],octa[4],zero,
    octa[4],octa[5],zero,
    octa[5],octa[6],zero,
    octa[6],octa[7],zero,
    octa[7],octa[0],zero
    
]

#pico
deca = f(0, 1/64, 10)
pico = (0,1/4,0)

objeto_3 = [
    deca[0], pico, deca[1],
    deca[1], pico, deca[2],
    deca[2], pico, deca[3],
    deca[3], pico, deca[4],
    deca[4], pico, deca[5],
    deca[5], pico, deca[6],
    deca[6], pico, deca[7],
    deca[7], pico, deca[8],
    deca[8], pico, deca[9],
    deca[9], pico, deca[0],

    deca[0], zero, deca[1],
    deca[1], zero, deca[2],
    deca[2], zero, deca[3],
    deca[3], zero, deca[4],
    deca[4], zero, deca[5],
    deca[5], zero, deca[6],
    deca[6], zero, deca[7],
    deca[7], zero, deca[8],
    deca[8], zero, deca[9],
    deca[9], zero, deca[0],
    
]

def esfera(raio, num_aneis, num_segmentos):
    vertices = []
    triangulos = []

    # Polo superior
    polo_superior = (0, raio, 0)
    vertices.append(polo_superior)

    # Gerar os vértices para cada anel de latitude
    for i in range(1, num_aneis):
        # Ângulo polar (latitude), variando entre 0 e pi
        theta = np.pi * i / num_aneis
        y = raio * np.cos(theta)
        r = raio * np.sin(theta)

        # Vértices ao longo deste anel de longitude
        for j in range(num_segmentos):
            phi = 2 * np.pi * j / num_segmentos
            x = r * np.cos(phi)
            z = r * np.sin(phi)
            vertices.append((x, y, z))

    # Polo inferior
    polo_inferior = (0, -raio, 0)
    vertices.append(polo_inferior)

    # Criar triângulos conectando os anéis
    # Triângulos do polo superior para o primeiro anel
    for j in range(num_segmentos):
        proximo = (j + 1) % num_segmentos
        triangulos.extend([polo_superior, vertices[1 + j], vertices[1 + proximo]])

    # Triângulos entre anéis consecutivos
    for i in range(1, num_aneis - 1):
        for j in range(num_segmentos):
            proximo = (j + 1) % num_segmentos
            p1 = 1 + (i - 1) * num_segmentos + j
            p2 = 1 + (i - 1) * num_segmentos + proximo
            p3 = 1 + i * num_segmentos + j
            p4 = 1 + i * num_segmentos + proximo

            # Adicionar os vértices dos triângulos sem agrupar de 3 em 3
            triangulos.extend([vertices[p1], vertices[p3], vertices[p4]])
            triangulos.extend([vertices[p1], vertices[p4], vertices[p2]])

    # Triângulos do último anel para o polo inferior
    ultimo_anel_inicio = 1 + (num_aneis - 2) * num_segmentos
    for j in range(num_segmentos):
        proximo = (j + 1) % num_segmentos
        triangulos.extend([vertices[ultimo_anel_inicio + j], polo_inferior, vertices[ultimo_anel_inicio + proximo]])

    return vertices, triangulos


vertices_esfera, objeto_4 = esfera(0.1, 16, 16) 

n_vertices_por_circulo = 32
circ_interno = f(0, 1/10, n_vertices_por_circulo)
circ_externo = f(0, 1/10+0.01 , n_vertices_por_circulo)

objeto_5 = []
for i in range(n_vertices_por_circulo):
    objeto_5.append(circ_interno[i])
    objeto_5.append(circ_externo[i])
    objeto_5.append(circ_interno[(i+1)%n_vertices_por_circulo])

    objeto_5.append(circ_interno[(i+1)%n_vertices_por_circulo])
    objeto_5.append(circ_externo[i])
    objeto_5.append(circ_externo[(i+1)%n_vertices_por_circulo])


objetos = []
objetos.append(objeto_1) #diamante
objetos.append(objeto_2) # cubo
objetos.append(objeto_3) # espinho/cone
objetos.append(objeto_4) # esfera
objetos.append(objeto_5) # anel
#------------------------------------------------NÃO EDITAR DAQUI PRA BAIXO.
#cria uma lista com todos os vértices
vertices__ = []
for obj in objetos:
    for coord in obj:
        vertices__.append(coord)
qtd_vertices = len(vertices__)


vertices = np.zeros(len(vertices__), [("position", np.float32, 3)])
vertices['position'] = vertices__

#matrizes base
def m_i():
    return np.array([   1.0,  0.0, 0.0, 0.0, 
                        0.0,  1.0, 0.0, 0.0, 
                        0.0,  0.0, 1.0, 0.0, 
                        0.0,  0.0, 0.0, 1.0], np.float32)

def m_tr(tx,ty,tz):
    return np.array([      1.0,  0.0, 0.0, tx, 
                           0.0,  1.0, 0.0, ty, 
                           0.0,  0.0, 1.0, tz, 
                           0.0,  0.0, 0.0, 1.0], np.float32)
def m_rot_z(t):
    return np.array([     math.cos(t), -math.sin(t),   0.0, 0.0, 
                          math.sin(t),  math .cos(t),  0.0, 0.0, 
                          0.0,          0.0,           1.0, 0.0, 
                          0.0,          0.0,           0.0, 1.0], np.float32)

def m_rot_x(t):
    return np.array([     1.0,   0.0,        0.0,         0.0, 
                          0.0, math.cos(t), -math.sin(t), 0.0, 
                          0.0, math.sin(t),  math.cos(t), 0.0, 
                          0.0,   0.0,        0.0,         1.0], np.float32)

def m_rot_y(t):
    return np.array([     math.cos(t),  0.0, math.sin(t), 0.0, 
                          0.0,          1.0,   0.0,       0.0, 
                          -math.sin(t), 0.0, math.cos(t), 0.0, 
                          0.0,          0.0,   0.0,       1.0], np.float32)

def m_s(sx,sy,sz):
    return np.array([          sx,  0.0, 0.0, 0.0, 
                              0.0,  sy, 0.0, 0.0, 
                                    0.0,  0.0, sz, 0.0, 
                                    0.0,  0.0, 0.0, 1.0], np.float32)



glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(900, 700, "CG2024 - Projeto 1", None, None)
glfw.make_context_current(window)


vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)


# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)


# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)


glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

loc_color = glGetUniformLocation(program, "color")


In [3]:
#controles
opcao_atual = 0
opcao_escolhida = 0
escolheu = False


def key_event(window,key,scancode,action,mods):
    global opcao_atual, escolheu, opcao_escolhida
    
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    
    if action == 1:
        if key == 263:
            opcao_atual -= 1   #seta esquerda
            opcao_atual = opcao_atual%3
        if key == 262: 
            opcao_atual += 1   #seta direita
            opcao_atual = opcao_atual%3
        if key == 257:
            opcao_escolhida = opcao_atual
            
            escolheu = not escolheu
        if key == 256:
            escolheu = False
    if action == 0:
        pass
    
glfw.set_key_callback(window,key_event)

glfw.show_window(window)

#------------------------------------------------------------------------------- NÂO EDITAR DAQUI PRA BAIXO.
import math
import random
d = 0.0
glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c



def renderizar(id_objeto, matriz_transformacao, R=3/4, G=3/4, B= 3/4):
    if id_objeto == 0: #cubo
        primeiro_vertice = 0
        qtd_vertices = len(objeto_1)
        glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_transformacao)
        
        for triangle in range(0, qtd_vertices, 3):
            random.seed(triangle)            
            glUniform4f(loc_color, R, G, B, 1.0)
            glDrawArrays(GL_TRIANGLES, triangle+primeiro_vertice, 3) 

    elif id_objeto == 1: #diamante
        primeiro_vertice = len(objeto_1)
        qtd_vertices = len(objeto_2)
        glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_transformacao)
        
        for triangle in range(0, qtd_vertices, 3):
            random.seed(triangle)            
            R = 0.6+ random.random()*0.6 - 0.6
            G = 0.7+ random.random()*0.6 - 0.3
            B = 1
            glUniform4f(loc_color, R, G, B, 1.0)
            glDrawArrays(GL_TRIANGLES, triangle+primeiro_vertice, 3)   

    elif id_objeto == 2:#espinho/cone
        primeiro_vertice = len(objeto_2)+len(objeto_1)
        qtd_vertices = len(objeto_3)
        glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_transformacao)
        
        for triangle in range(0, qtd_vertices, 3):
            random.seed(triangle)            
            R = random.random()
            G = random.random()
            B = random.random()
            glUniform4f(loc_color, R, G, B, 1.0)
            glDrawArrays(GL_TRIANGLES, triangle+primeiro_vertice, 3)   
    elif id_objeto == 3: #esfera
        primeiro_vertice = len(objeto_2)+len(objeto_1)+len(objeto_3)
        qtd_vertices = len(objeto_4)
        glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_transformacao)
        
        for triangle in range(0, qtd_vertices, 3):
            random.seed(triangle)            
            R = random.random()
            G = random.random()
            B = random.random()
            glUniform4f(loc_color, R, G, B, 1.0)
            glDrawArrays(GL_TRIANGLES, triangle+primeiro_vertice, 3)   
    elif id_objeto ==4: #anel
        primeiro_vertice = len(objeto_2)+len(objeto_1)+len(objeto_3)+len(objeto_4)
        qtd_vertices = len(objeto_5)
        glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_transformacao)
        
        for triangle in range(0, qtd_vertices, 3):
            random.seed(triangle)            
            R = random.random()
            G = random.random()
            B = random.random()
            glUniform4f(loc_color, R, G, B, 1.0)
            glDrawArrays(GL_TRIANGLES, triangle+primeiro_vertice, 3)   
        
    pass
        

        
    
        
        


def matriz_transform(tx, ty, tz, r_x, r_y, r_z, s_x, s_y, s_z):
    mat_translate = m_tr(tx, ty, tz)
    mat_rot_z = m_rot_z(r_z)
    mat_rot_x = m_rot_x(r_x)
    mat_rot_y = m_rot_y(r_y)
    mat_scale = m_s(s_x, s_y, s_z)
    
    mat_transform = multiplica_matriz(mat_rot_x, mat_rot_y) 
    mat_transform = multiplica_matriz(mat_transform, mat_rot_z) 
    mat_transform = multiplica_matriz(mat_transform,mat_scale)
    # print(mat_transform)
    mat_transform = multiplica_matriz(mat_transform, mat_translate)
        
    return mat_transform

def mult_n_matrizes(array_matrizes):
    '''
    É importante conferir que as matrizes são multiplicadas da esquerda pra direita e são aplicadas pela direita
    '''
    # Verifica se o array de matrizes não está vazio
    if len(array_matrizes) == 0:
        print("O array de matrizes está vazio")
        raise ValueError("O array de matrizes está vazio.")
    
    # Inicializa o resultado com a primeira matriz e garante que seja 4x4
    resultado = np.atleast_2d(array_matrizes[0]).reshape(4, 4)
    
    # Multiplica todas as matrizes subsequentes
    for matriz in array_matrizes[1:]:
        matriz = np.atleast_2d(matriz).reshape(4, 4)  # Garante que cada matriz seja 4x4
        
        resultado = np.matmul(resultado, matriz)
    
    return resultado



counter = 0
while not glfw.window_should_close(window):
    
    cont = 0
    glfw.poll_events() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(0.0,0.0, 0.0, 1.0)
    
    loc = glGetUniformLocation(program, "mat_transformation")
    # -----------------------------
    
    dt = counter*np.pi/1800

    #mesa
    m_m1 = mult_n_matrizes([m_s(1.5, 1/16, 1.5)])
    m_m2 = mult_n_matrizes([m_tr(-1/2, -1/2,  -1/2),  m_s(1/16, 1/2, 1/16)])
    m_m3 = mult_n_matrizes([m_tr(-1/2, -1/2,   1/2),  m_s(1/16, 1/2, 1/16)])
    m_m4 = mult_n_matrizes([m_tr(1/2,  -1/2,  -0.5),  m_s(1/16, 1/2, 1/16)])
    m_m5 = mult_n_matrizes([m_tr(1/2,  -1/2,   0.5),  m_s(1/16, 1/2, 1/16)])

    #mão
    seno = 0+np.sin(21*dt)/50
    
    if opcao_atual == 0:
        m_pos = m_tr(-1/3,-1/4,-1/3)
    elif opcao_atual == 1:
        m_pos = m_tr(0,-1/4,-1/3)
    elif opcao_atual == 2:
        m_pos = m_tr(1/3,-1/4,-1/3)
        
    m_h1 = mult_n_matrizes([m_tr(0, 1/2+seno ,-29/64) ,  m_s(1/16, 1/16, 4/12)])
    m_h2 = mult_n_matrizes([m_tr(0, 1/2+seno ,-1/4) ,    m_s(1/4, 1/16, 1/4)])
    
    m_h3 = mult_n_matrizes([m_tr(-3/32, 33/64+seno ,-2/16) ,m_s(1/32, 1/32, 1/8)])
    m_h4 = mult_n_matrizes([m_tr(-1/32, 33/64+seno ,-2/16) ,m_s(1/32, 1/32, 1/8)])
    m_h5 = mult_n_matrizes([m_tr(1/32,  33/64+seno ,-2/16)  ,m_s(1/32, 1/32, 1/8)])
    m_h6 = mult_n_matrizes([m_tr(3/32,  33/64+seno ,-2/16)  ,m_s(1/32, 1/32, 1/8)])

    m_h7 = mult_n_matrizes([m_tr(-3/32, 33/64+seno ,-1/16) ,                  m_s(1/32, 1/32, 1/8)])
    m_h8 = mult_n_matrizes([m_tr(-1/32, 31/64+seno ,-1/16), m_rot_x(np.pi/4), m_s(1/32, 1/32, 1/8)])
    m_h9 = mult_n_matrizes([m_tr(1/32,  31/64+seno , -1/16)  ,m_rot_x(np.pi/4), m_s(1/32, 1/32, 1/8)])
    m_h10 = mult_n_matrizes([m_tr(3/32, 31/64+seno ,-1/16)  ,m_rot_x(np.pi/4), m_s(1/32, 1/32, 1/8) ])
    
    m_h11 = mult_n_matrizes([m_tr(-7/64, 33/64+seno ,-4/16) ,m_rot_y(-np.pi/6),m_s(1/32, 1/32, 1/8)])
    m_h12 = mult_n_matrizes([m_tr(-9/64, 33/64+seno ,-5/32) , m_s(1/32, 1/32, 1/8)])
    
    #diamante
    m_d2 = mult_n_matrizes([m_tr(0,0,1/3)   , m_tr(0,1/16,0), m_rot_y(-8*dt), m_s(1/5,1/5,1/5)])
    
    
    #mamona
    # espinhos direto no plot #passar pra cima (não deve ficar aqui) #implementar
    # (...)
    m_e = mult_n_matrizes([ m_i()  ])
    m_mamona = mult_n_matrizes([m_tr(0,0,1/3), m_tr(1/3,0,0) , m_rot_y(-5*dt), m_s(12/20,12/20,12/20)])

    #planeta
    m_e2 = mult_n_matrizes([m_rot_y(-20*dt),m_s(1/2,1/2,1/2)])
    m_anel = mult_n_matrizes([m_rot_z(5*dt),m_rot_y(30*dt), m_s(1.1,1,1)])
    m_planeta = mult_n_matrizes([m_tr(-1/3,1/3,0),m_s(1,1,1)])
    
    #----
    
    #inclinação é aplicada à todos
    m_inclina = mult_n_matrizes([m_tr(0,-1/5,0) ,m_rot_x(-20*np.pi/180), m_rot_y(np.sin(1*dt)*10 * np.pi/180), m_rot_x(np.sin(5*dt)*10 * np.pi/180)])

        
    if escolheu:
        print("escolhido:",opcao_escolhida)
        m_mamona = m_tr(-10,0,0)
        m_d2 = m_tr(-10,0,0)
        m_planeta = m_tr(-10,0,0)
        m_pos = m_tr(-10,0,0)
        
        if opcao_escolhida == 0:
            m_planeta = mult_n_matrizes([m_tr(0,2/3,0),m_s(2,2,2)])
        if opcao_escolhida == 1:
            m_d2 = mult_n_matrizes ([m_tr(0,1/3,0),m_rot_y(5*dt), m_s(3/5,3/5,3/5)] )
        if opcao_escolhida == 2:
            m_mamona = mult_n_matrizes ( [m_tr(0,1/3,0),m_rot_y(5*dt), m_s(1,1,1)] )
            
    else:
        m_planeta = mult_n_matrizes( [m_tr(-1/3,1/3,0),m_s(1,1,1)] )
        m_mamona = mult_n_matrizes( [m_tr(0,0,1/3), m_tr(1/3,0,0) , m_rot_y(-5*dt), m_s(12/20,12/20,12/20)] )
        m_d2 = mult_n_matrizes( [m_tr(0,0,1/3), m_tr(0,1/16,0), m_rot_y(-8*dt), m_s(1/5,1/5,1/5)] )
        #m_pos atualiza sozinha
            




# -------------------------------------------------------

    random.seed()
    
    
    #diamante
    renderizar(1, mult_n_matrizes([m_inclina, m_d2])   )
    

    #mesa
    renderizar(0, mult_n_matrizes([m_inclina ,m_m1]), R=0.2, G=0,B=0)
    renderizar(0, mult_n_matrizes([m_inclina ,m_m2]), R=0.2, G=0,B=0)
    renderizar(0, mult_n_matrizes([m_inclina ,m_m3]), R=0.2, G=0,B=0)
    renderizar(0, mult_n_matrizes([m_inclina ,m_m4]), R=0.2, G=0,B=0)
    renderizar(0, mult_n_matrizes([m_inclina ,m_m5]), R=0.2, G=0,B=0)

    #mão
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h1]),R = 13/24)
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h2]),R = 14/24)
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h3]),R = 0/24, G= 0, B=0)
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h4]),R = 16/24)
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h5]),R = 17/24)
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h6]),R = 18/24)
    
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h7]),R = 19/24)
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h8]),R = 20/24)
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h9]),R = 21/24)
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h10]),R = 22/24)

    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h11]),R = 23/24)
    renderizar(0, mult_n_matrizes([m_pos, m_inclina ,m_h12]),R = 24/24)

    #mamona
    #espinhos #implementar, corrigir isso, não deve ser aqui.
    pos_mamona = m_tr(0,1/3,0)
    for i in range(0,16):
        renderizar(2, mult_n_matrizes( [ m_inclina ,m_mamona, pos_mamona, m_rot_x(i*np.pi/8) ]) ) 
    for i in range(0,16):
        renderizar(2, mult_n_matrizes( [ m_inclina , m_mamona, pos_mamona, m_rot_y(i*np.pi/8) ]) ) 
    for i in range(0,16):
        renderizar(2, mult_n_matrizes( [ m_inclina , m_mamona, pos_mamona, m_rot_z(i*np.pi/8) ]) ) 
    #esfera
    renderizar(3, mult_n_matrizes([m_inclina ,m_mamona, m_tr(0,1/3,0), m_e] ))


    #planeta
    renderizar(3,mult_n_matrizes([m_inclina, m_planeta, m_e2])) 
    renderizar(4,mult_n_matrizes([m_inclina, m_planeta, m_anel]))
    #----------------------------------------------------------------------------


    
    glfw.swap_buffers(window)
    counter += 1

glfw.terminate()

